In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
seed = 2
from xgboost import XGBClassifier
import pickle

In [2]:
train = pd.read_csv('./data/new_train.csv')

In [ ]:
sample_size = int(1e4)
sample = train[train.target == 1].sample(n=sample_size/2, random_state=seed)
sample = sample.append( train[train.target == 0].sample(n=sample_size/2, random_state=seed))
sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)

In [3]:
target = train.target
train.drop(['target','id'], inplace=True, axis=1)
train.drop([ col for col in train.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [4]:
with open('./data/OneHotEncoder.clf', 'rb') as f:
    encoders = pickle.load(f)

In [6]:
enc_train = None

for feature,encoder in zip(train.columns,encoders) :
    encoded = encoder.transform(train[feature].values.reshape(-1,1))
    if enc_train is None :
        enc_train = encoded
    else :
        enc_train = np.concatenate((enc_train, encoded), axis=1)

In [ ]:
del train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(enc_train, target, test_size=.2, random_state=seed)

In [7]:
model = XGBClassifier(
	objective = 'binary:logistic',
	n_jobs = 4,
	random_state = seed,
	eval_metric = "error",
	#-------
	max_depth = 1,
	min_child_weight = 2,
	#----------
	gamma = 0,
	#----------
	subsample = 0.1,
	colsample_bytree = 0.1,
	#------------
	scale_pos_weight = 1,
	#-----------
	reg_alpha = 0,
	reg_lambda = 0,
	#------------
	learning_rate = 0.1,

	#n_estimators = 1000,
	)

In [8]:
model.fit(enc_train, target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.1, eval_metric='error', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=1,
       min_child_weight=2, missing=None, n_estimators=100, n_jobs=4,
       nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.1)

In [9]:
with open('./data/model.pkl', 'wb') as f:
    pickle.dump(file=f, obj=model)

In [11]:
enc_train.shape

(595212, 385)

In [ ]:
encoder.fit_transform(train.ps_cat_ind_02.values.reshape(-1,1))

In [ ]:
train.ps_cat_ind_02.value_counts()

In [10]:
train.shape


(595212, 52)